In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
生成图片浏览 HTML（缩略图网格 + 灯箱查看），并支持：
- 大图滚轮缩放（以鼠标位置为锚点）
- 拖拽平移
- 按钮：放大/缩小/适配/1:1
- 双击：适配 <-> 1:1 切换
- 键盘：左右切换、Esc 关闭、+/- 缩放、F/0 适配
- 点击黑色背景关闭（包含：屏幕 backdrop + stage 内空白黑底）

用法：
  python gallery.py /path/to/images -o /path/to/out/gallery.html -t "My Gallery"
默认输出：
  不指定 -o 时输出到图片目录下 gallery.html
"""

from pathlib import Path
import html
import mimetypes
import argparse

IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp", ".gif", ".bmp", ".svg"}

HTML_TEMPLATE = """<!doctype html>
<html lang="zh-CN">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>{title}</title>
  <style>
    :root {{
      --gap: 12px;
      --thumb-h: 180px;
      --bg: #0b0c10;
      --card: #111218;
      --text: #e8e8ea;
      --muted: #a9abb3;
      --border: rgba(255,255,255,0.10);
    }}
    body {{
      margin: 0;
      font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, "Helvetica Neue", Arial;
      background: var(--bg);
      color: var(--text);
    }}
    header {{
      position: sticky;
      top: 0;
      backdrop-filter: blur(8px);
      background: rgba(11,12,16,0.75);
      border-bottom: 1px solid var(--border);
      padding: 12px 16px;
      z-index: 10;
    }}
    .title {{
      font-size: 16px;
      font-weight: 600;
      margin: 0 0 6px 0;
    }}
    .meta {{
      font-size: 12px;
      color: var(--muted);
      display: flex;
      gap: 14px;
      flex-wrap: wrap;
    }}
    .wrap {{
      padding: 16px;
      max-width: 1400px;
      margin: 0 auto;
    }}
    .grid {{
      display: grid;
      grid-template-columns: repeat(auto-fill, minmax(220px, 1fr));
      gap: var(--gap);
    }}

    /* Card as button */
    .cardbtn {{
      appearance: none;
      border: none;
      background: transparent;
      padding: 0;
      text-align: left;
      cursor: zoom-in;
      color: inherit;
    }}

    .card {{
      background: var(--card);
      border: 1px solid var(--border);
      border-radius: 14px;
      overflow: hidden;
      box-shadow: 0 6px 20px rgba(0,0,0,0.25);
      transition: transform .12s ease;
    }}
    .cardbtn:hover .card {{ transform: translateY(-2px); }}

    .thumb {{
      height: var(--thumb-h);
      display: flex;
      align-items: center;
      justify-content: center;
      background: #0f1016;
    }}
    .thumb img {{
      max-width: 100%;
      max-height: 100%;
      object-fit: contain;
      display: block;
      user-select: none;
      -webkit-user-drag: none;
    }}
    .info {{
      padding: 10px 12px 12px 12px;
      border-top: 1px solid var(--border);
    }}
    .name {{
      font-size: 12px;
      line-height: 1.35;
      word-break: break-all;
    }}
    .tag {{
      margin-top: 6px;
      font-size: 11px;
      color: var(--muted);
    }}

    .footer {{
      padding: 18px 0 10px 0;
      text-align: center;
      color: var(--muted);
      font-size: 12px;
    }}

    /* Lightbox */
    .lightbox {{
      position: fixed;
      inset: 0;
      display: none;
      z-index: 9999;
    }}
    .lightbox.open {{ display: block; }}
    .lb-backdrop {{
      position: absolute;
      inset: 0;
      background: rgba(0,0,0,0.75);
    }}
    .lb-panel {{
      position: absolute;
      inset: 24px;
      display: grid;
      grid-template-rows: auto 1fr auto;
      gap: 10px;
    }}
    .lb-top {{
      display: flex;
      justify-content: space-between;
      align-items: center;
      gap: 12px;
      color: var(--text);
      font-size: 13px;
    }}
    .lb-title {{
      overflow: hidden;
      text-overflow: ellipsis;
      white-space: nowrap;
      color: var(--muted);
    }}
    .lb-actions {{
      display: flex;
      gap: 8px;
      flex-shrink: 0;
    }}
    .lb-btn {{
      border: 1px solid var(--border);
      background: rgba(17,18,24,0.8);
      color: var(--text);
      border-radius: 10px;
      padding: 8px 10px;
      cursor: pointer;
      font-size: 12px;
    }}
    .lb-btn:hover {{ filter: brightness(1.1); }}

    .lb-stage {{
      position: relative;
      border: 1px solid var(--border);
      background: rgba(17,18,24,0.5);
      border-radius: 16px;
      overflow: hidden;
      display: flex;
      align-items: center;
      justify-content: center;
      touch-action: none; /* 允许我们接管滚轮/拖拽 */
    }}

    /* 大图：用 CSS 变量控制缩放/平移 */
    .lb-img{{
      --tx: 0px;
      --ty: 0px;
      --s: 1;

      max-width: none;
      max-height: none;
      object-fit: contain;

      position: absolute;
      left: 50%;
      top: 50%;
      transform: translate(calc(-50% + var(--tx)), calc(-50% + var(--ty))) scale(var(--s));
      transform-origin: center center;

      user-select: none;
      -webkit-user-drag: none;
      cursor: grab;
    }}
    .lb-stage.grabbing .lb-img {{ cursor: grabbing; }}

    .lb-bottom {{
      display: flex;
      justify-content: center;
      gap: 10px;
      color: var(--muted);
      font-size: 12px;
    }}
    .lb-hint {{
      opacity: 0.9;
    }}
    body.lb-lock {{
      overflow: hidden;
    }}
  </style>
</head>
<body>
<header>
  <div class="title">{title}</div>
  <div class="meta">
    <div>目录：{dir_name}</div>
    <div>图片数：{count}</div>
  </div>
</header>

<div class="wrap">
  <div class="grid">
    {cards}
  </div>
  <div class="footer">Generated by Python</div>
</div>

<!-- Lightbox -->
<div id="lightbox" class="lightbox" aria-hidden="true">
  <div class="lb-backdrop" id="lb-backdrop"></div>
  <div class="lb-panel" role="dialog" aria-modal="true" aria-label="Image viewer">
    <div class="lb-top">
      <div class="lb-title" id="lb-title"></div>
      <div class="lb-actions">
        <button class="lb-btn" id="lb-zoomout" type="button">−</button>
        <button class="lb-btn" id="lb-zoomin" type="button">+</button>
        <button class="lb-btn" id="lb-fit" type="button">适配</button>
        <button class="lb-btn" id="lb-actual" type="button">1:1</button>

        <button class="lb-btn" id="lb-prev" type="button">← 上一张</button>
        <button class="lb-btn" id="lb-next" type="button">下一张 →</button>
        <button class="lb-btn" id="lb-close" type="button">关闭 (Esc)</button>
      </div>
    </div>

    <div class="lb-stage">
      <img id="lb-img" class="lb-img" alt="">
    </div>

    <div class="lb-bottom">
      <div class="lb-hint">提示：滚轮缩放，拖拽平移，双击适配/1:1，左右键切换，Esc 关闭，点击黑色背景关闭</div>
    </div>
  </div>
</div>

<script>
(function() {{
  const items = Array.from(document.querySelectorAll('[data-lb="item"]'));
  const lightbox = document.getElementById('lightbox');
  const imgEl = document.getElementById('lb-img');
  const titleEl = document.getElementById('lb-title');

  const btnClose = document.getElementById('lb-close');
  const btnPrev = document.getElementById('lb-prev');
  const btnNext = document.getElementById('lb-next');
  const backdrop = document.getElementById('lb-backdrop');

  const btnZoomIn = document.getElementById('lb-zoomin');
  const btnZoomOut = document.getElementById('lb-zoomout');
  const btnFit = document.getElementById('lb-fit');
  const btnActual = document.getElementById('lb-actual');

  const stageEl = document.querySelector('.lb-stage');

  let idx = -1;

  // ===== Zoom/Pan state =====
  let scale = 1;
  let tx = 0;
  let ty = 0;
  let fitScale = 1;
  let userInteracted = false;

  const MIN_SCALE = 0.05;
  const MAX_SCALE = 16;

  function applyTransform() {{
    imgEl.style.setProperty('--s', String(scale));
    imgEl.style.setProperty('--tx', `${{tx}}px`);
    imgEl.style.setProperty('--ty', `${{ty}}px`);
  }}

  function clamp(v, a, b) {{ return Math.max(a, Math.min(b, v)); }}

  function fitToStage() {{
    const iw = imgEl.naturalWidth || 1;
    const ih = imgEl.naturalHeight || 1;

    const sw = stageEl.clientWidth || 1;
    const sh = stageEl.clientHeight || 1;

    fitScale = Math.min(sw / iw, sh / ih);
    fitScale = clamp(fitScale, MIN_SCALE, MAX_SCALE);

    scale = fitScale;
    tx = 0;
    ty = 0;
    userInteracted = false;
    applyTransform();
  }}

  function setActual() {{
    scale = 1;
    tx = 0;
    ty = 0;
    userInteracted = true;
    applyTransform();
  }}

  function zoomTo(newScale, clientX, clientY) {{
    newScale = clamp(newScale, MIN_SCALE, MAX_SCALE);

    const rect = stageEl.getBoundingClientRect();
    const cx = rect.left + rect.width / 2;
    const cy = rect.top + rect.height / 2;

    // 当前鼠标点在“图像坐标系(以图片中心为原点)”下的位置
    const dx = (clientX - cx - tx) / scale;
    const dy = (clientY - cy - ty) / scale;

    // 更新 scale 后，反推 tx/ty，让鼠标点尽量保持不动
    scale = newScale;
    tx = (clientX - cx) - dx * scale;
    ty = (clientY - cy) - dy * scale;

    userInteracted = true;
    applyTransform();
  }}

  function zoomBy(factor, anchorX, anchorY) {{
    zoomTo(scale * factor, anchorX, anchorY);
  }}

  function zoomAtCenter(factor) {{
    const rect = stageEl.getBoundingClientRect();
    zoomBy(factor, rect.left + rect.width / 2, rect.top + rect.height / 2);
  }}

  // ===== Lightbox navigation =====
  function setIndex(newIdx) {{
    if (!items.length) return;
    idx = (newIdx + items.length) % items.length;

    const el = items[idx];
    const src = el.getAttribute('data-src');
    const name = el.getAttribute('data-name') || '';

    imgEl.removeAttribute('src');
    imgEl.onload = () => {{
      fitToStage(); // 每张图打开默认适配
    }};

    imgEl.src = src;
    imgEl.alt = name;
    titleEl.textContent = `${{idx + 1}} / ${{items.length}}  ·  ${{name}}`;
  }}

  function openAt(i) {{
    setIndex(i);
    lightbox.classList.add('open');
    lightbox.setAttribute('aria-hidden', 'false');
    document.body.classList.add('lb-lock');

    stageEl.classList.remove('grabbing');
  }}

  function close() {{
    lightbox.classList.remove('open');
    lightbox.setAttribute('aria-hidden', 'true');
    document.body.classList.remove('lb-lock');

    imgEl.removeAttribute('src');

    scale = 1; tx = 0; ty = 0; fitScale = 1; userInteracted = false;
    applyTransform();
    stageEl.classList.remove('grabbing');
  }}

  function prev() {{ setIndex(idx - 1); }}
  function next() {{ setIndex(idx + 1); }}

  // ===== Bind card click =====
  items.forEach((el, i) => {{
    el.addEventListener('click', () => openAt(i));
  }});

  // ===== Buttons =====
  btnClose.addEventListener('click', close);
  backdrop.addEventListener('click', close);

  btnPrev.addEventListener('click', prev);
  btnNext.addEventListener('click', next);

  btnFit.addEventListener('click', fitToStage);
  btnActual.addEventListener('click', setActual);
  btnZoomIn.addEventListener('click', () => zoomAtCenter(1.25));
  btnZoomOut.addEventListener('click', () => zoomAtCenter(0.8));

  // ===== Wheel zoom =====
  stageEl.addEventListener('wheel', (e) => {{
    if (!lightbox.classList.contains('open')) return;
    e.preventDefault();

    const factor = (e.deltaY > 0) ? 0.9 : 1.1;
    zoomBy(factor, e.clientX, e.clientY);
  }}, {{ passive: false }});

  // ===== Drag to pan (Pointer Events) + 点击黑色空白关闭 =====
  let dragging = false;
  let startX = 0, startY = 0;
  let startTx = 0, startTy = 0;

  // 新增：拖拽判定 + 防误触关闭
  let moved = false;
  let lastDragAt = 0;

  stageEl.addEventListener('pointerdown', (e) => {{
    if (!lightbox.classList.contains('open')) return;

    dragging = true;
    moved = false;
    startX = e.clientX;
    startY = e.clientY;
    startTx = tx;
    startTy = ty;

    stageEl.classList.add('grabbing');
    stageEl.setPointerCapture(e.pointerId);
  }});

  stageEl.addEventListener('pointermove', (e) => {{
    if (!dragging) return;

    const dx = e.clientX - startX;
    const dy = e.clientY - startY;

    // 如果移动超过阈值，认为是拖拽
    if (Math.abs(dx) + Math.abs(dy) > 2) moved = true;

    tx = startTx + dx;
    ty = startTy + dy;

    userInteracted = true;
    applyTransform();
  }});

  stageEl.addEventListener('pointerup', (e) => {{
    if (!dragging) return;
    dragging = false;
    stageEl.classList.remove('grabbing');
    try {{ stageEl.releasePointerCapture(e.pointerId); }} catch (_) {{}}

    // 记录拖拽结束时间，避免拖完立刻 click 触发关闭
    if (moved) lastDragAt = Date.now();
  }});

  stageEl.addEventListener('pointercancel', (e) => {{
    dragging = false;
    stageEl.classList.remove('grabbing');
    try {{ stageEl.releasePointerCapture(e.pointerId); }} catch (_) {{}}
  }});

  // 点击 stage 的空白区域关闭（点到图片不关）
  stageEl.addEventListener('click', (e) => {{
    if (!lightbox.classList.contains('open')) return;
    if (Date.now() - lastDragAt < 250) return; // 刚拖完不关
    if (e.target === stageEl) close();
  }});

  // ===== Double click: toggle fit <-> 1:1 =====
  stageEl.addEventListener('dblclick', (e) => {{
    if (!lightbox.classList.contains('open')) return;

    if (Math.abs(scale - fitScale) < 1e-3) {{
      zoomTo(1, e.clientX, e.clientY); // 以双击点为锚点放到 1:1
    }} else {{
      fitToStage();
    }}
  }});

  // ===== Keyboard =====
  window.addEventListener('keydown', (e) => {{
    if (!lightbox.classList.contains('open')) return;

    if (e.key === 'Escape') close();
    else if (e.key === 'ArrowLeft') prev();
    else if (e.key === 'ArrowRight') next();
    else if (e.key === '+' || e.key === '=') zoomAtCenter(1.25);
    else if (e.key === '-' || e.key === '_') zoomAtCenter(0.8);
    else if (e.key.toLowerCase() === 'f') fitToStage();
    else if (e.key === '0') fitToStage();
  }});

  // ===== Resize: 如果用户还没手动缩放/拖拽，就保持适配 =====
  window.addEventListener('resize', () => {{
    if (!lightbox.classList.contains('open')) return;
    if (!userInteracted && imgEl.getAttribute('src')) {{
      fitToStage();
    }}
  }});
}})();
</script>

</body>
</html>
"""

CARD_TEMPLATE = """
<button type="button" class="cardbtn" data-lb="item" data-src="{src}" data-name="{name}">
  <div class="card">
    <div class="thumb">
      <img src="{src}" loading="lazy" alt="{alt}">
    </div>
    <div class="info">
      <div class="name">{name}</div>
      <div class="tag">{size}</div>
    </div>
  </div>
</button>
"""


def human_size(num_bytes: int) -> str:
    units = ["B", "KB", "MB", "GB", "TB"]
    size = float(num_bytes)
    i = 0
    while size >= 1024 and i < len(units) - 1:
        size /= 1024.0
        i += 1
    if i == 0:
        return f"{int(size)} {units[i]}"
    return f"{size:.2f} {units[i]}"


def is_image(p: Path) -> bool:
    if p.suffix.lower() in IMG_EXTS:
        return True
    mt, _ = mimetypes.guess_type(str(p))
    return (mt or "").startswith("image/")


def generate_gallery_html(img_dir: Path, out_html: Path, title: str | None = None) -> None:
    img_dir = img_dir.resolve()
    out_html = out_html.resolve()
    title = title or f"Image Gallery - {img_dir.name}"

    images = [p for p in img_dir.iterdir() if p.is_file() and is_image(p)]
    images.sort(key=lambda p: p.name.lower())

    cards = []
    for p in images:
        rel = p.relative_to(out_html.parent)
        rel_str = str(rel).replace("\\", "/")
        stat = p.stat()
        cards.append(
            CARD_TEMPLATE.format(
                src=html.escape(rel_str),
                alt=html.escape(p.name),
                name=html.escape(p.name),
                size=human_size(stat.st_size),
            )
        )

    out_html.parent.mkdir(parents=True, exist_ok=True)
    page = HTML_TEMPLATE.format(
        title=html.escape(title),
        dir_name=html.escape(str(img_dir)),
        count=len(images),
        cards="\n".join(cards),
    )
    out_html.write_text(page, encoding="utf-8")

In [8]:
img_dir = Path("E:\\project\\cloud_seg_compare_page\\20260126")
out_html = Path("20260126.html")
title = '20260126 Cloud Segmentation Results'

if not img_dir.exists() or not img_dir.is_dir():
    raise SystemExit(f"Not a directory: {img_dir}")

generate_gallery_html(img_dir, out_html, title=title)
print(f"✅ Generated: {out_html.resolve()}")


✅ Generated: E:\project\cloud_seg_compare_page\20260126.html
